In [3]:
import scvelo as scv
from IPython.display import clear_output
import matplotlib.backends.backend_pdf
from tqdm import tnrange, tqdm_notebook
import scanpy as sc
import matplotlib.pyplot as pl
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline


scv.logging.print_version()
scv.settings.set_figure_params(
    'scvelo', dpi_save=100, dpi=80, transparent=True)
scv.settings.verbosity = 2

Running scvelo 0.1.16.dev32+c00a55e.dirty (python 3.6.6) on 2020-12-08 14:24.


In [4]:
# paths and names
data_path='G:/data/scSLAMseq/revision/'
signatures_path='G:/data/scrnaseq_signature_collection/'
figure_path='G:/data/figures/heatmaps/'
libraries = ['AB', 'CE', 'DF']
donors=['B2-040', 'C2-019', 'OT227', 'OT302', 'P009T', 'P013T']

In [5]:
donor=donors[-1]
adata = scv.read(data_path+'by_donors/processed/SLAMv2_'+donor+'_processed.h5')

In [6]:
def restrict_KNN(adata):
    # restrict KNN connectivities to within perturbations only
    from scipy.sparse import csr_matrix
    adata = adata[np.argsort(adata.obs.perturbation)].copy()  # sort by perturbation
    A=adata.obsp['connectivities'].A
    # pl.imshow(A*5000, vmax=0.1)
    # pl.show()
    for pert in pd.unique(adata.obs.perturbation):
        a=np.where(adata.obs.perturbation==pert)[0]
        b=np.where(adata.obs.perturbation!=pert)[0]
        # remove neighborhoods
        A[np.min(a):np.max(a), b]=0
        A[b, np.min(a):np.max(a)]=0
    adata.obsp['connectivities'] = csr_matrix(A)
    # pl.imshow(A*5000, vmax=0.1)
    return adata

In [12]:
#del adata.obsm['X_diffmap']
adata.layers['unspliced']=adata.layers['new']
adata.layers['spliced']=adata.layers['old']
scv.pp.neighbors(adata)

computing neighbors
    finished (0:00:00)


In [13]:
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.velocity(adata, vkey='connected_velocity')
scv.tl.velocity_graph(adata, vkey='connected_velocity')
scv.tl.velocity_embedding(adata, basis='umap', vkey='connected_velocity')

computing moments based on connectivities


MemoryError: Unable to allocate 146. MiB for an array with shape (38257603,) and data type float32

In [ ]:
adata = restrict_KNN(adata)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
scv.tl.velocity(adata, vkey='disconnected_velocity')
scv.tl.velocity_graph(adata, vkey='disconnected_velocity')
scv.tl.velocity_embedding(adata, basis='umap', vkey='disconnected_velocity')

In [ ]:
scv.pl.velocity_embedding_stream(adata, vkey='connected_velocity', color=['perturbation', 'Enterocytes 1', 'phase', 'Stem'], title='connected_velocity')

In [ ]:
scv.pl.velocity_embedding_stream(adata, vkey='disconnected_velocity', color=['perturbation', 'Enterocytes 1', 'phase', 'Stem'], title='disconnected_velocity')